In [194]:
from IPython.display import HTML

HTML('''
<script>
  function code_toggle() {    if (code_shown){      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {$('div.input').show('500');$('#toggleButton').val('Hide Code')    }code_shown = !code_shown
  }  $( document ).ready(function(){code_shown=false;$('div.input').hide()
  });</script><form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
''')

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import re

import os
import json

In [24]:
def strip_exp_names(e, iden='FINAL_RESULTS'):
    i = e.find(iden)+len(iden)+1
    return e[i:-5]

def add_multicolumn(df, df2, new_col_name):
    tmp = df2.copy()    # make copy, otherwise df2 will be changed !!!
    tmp.columns = pd.MultiIndex.from_product([[new_col_name], df2.columns.tolist()])
    return pd.concat([df, tmp], axis=1)

def get_valid_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [60]:
model_path = '../attack_results/white_box/'
all_nets =  list(os.listdir(model_path))
all_nets = [a for a in all_nets if 'FINAL_RESULTS' in a]
# all_nets = [strip_exp_names(a) for a in all_nets]
# all_nets

In [179]:
all_nets

['white_box_attacks_FINAL_RESULTS_resnet56_cifar10.json',
 'white_box_attacks_FINAL_RESULTS_transfer_12_layers_resnet56_nat_pgd.json',
 'white_box_attacks_FINAL_RESULTS_transfer_12_layers_resnet56_nat_fgsm.json',
 'white_box_attacks_FINAL_RESULTS_transfer_12_layers_resnet56_fgsm_fgsm.json',
 'white_box_attacks_FINAL_RESULTS_transfer_feat_extractor_resnet56_nat_pgd.json',
 'white_box_attacks_FINAL_RESULTS_transfer_12_layers_resnet56_fgsm_nat.json',
 'white_box_attacks_FINAL_RESULTS_transfer_all_layers_resnet56_fgsm_fgsm.json',
 'white_box_attacks_FINAL_RESULTS_transfer_feat_extractor_resnet56_fgsm_fgsm.json',
 'white_box_attacks_FINAL_RESULTS_resnet56_cifar10_fgsm.json',
 'white_box_attacks_FINAL_RESULTS_transfer_all_layers_resnet56_nat_fgsm.json',
 'white_box_attacks_FINAL_RESULTS_transfer_feat_extractor_resnet56_nat_fgsm.json',
 'white_box_attacks_FINAL_RESULTS_transfer_all_layers_resnet56_nat_pgd.json',
 'white_box_attacks_FINAL_RESULTS_transfer_feat_extractor_resnet56_fgsm_nat.json'

In [61]:
nets_baselines = [a for a in all_nets if 'FINAL_RESULTS' in a and 'no_LL' not in a and 'all_layers' not in a and 'transfer' not in a
                        and 'cifar100' not in a]
nets_baselines.sort()
# nets_baselines

In [62]:
nets_transfer = [a for a in all_nets if 'FINAL_RESULTS' in a and 'transfer' in a]
nets_transfer.sort()
# nets_transfer

In [173]:
nets = all_nets

attack_results = {}

for net in nets:
    fname = model_path+net
    with open(fname,'r+') as json_data:
        attack_results[strip_exp_names(net)] = list(json.load(json_data).values())
        
df = pd.DataFrame.from_dict(attack_results, columns=['Clean', 
                                                     'FGSM 0.03125', 'PGD 0.03125',
                                                     'FGSM 0.0625', 'PGD 0.0625',
                                                    ], orient='index')

for row in df.iterrows():
    s = row[0].split('_')
#     print(s)
    if s[0] == 'transfer':
#         df.loc[row[0], 'Net'] = s[3]
        df.loc[row[0], 'Type'] = 'Transfer'
        df.loc[row[0], 'Source Adversary'] = s[4]
        df.loc[row[0], 'Target Adversary'] = s[5]
        df.loc[row[0], 'Transfer'] = s[1] + ' ' + s[2]
    else:
#         df.loc[row[0], 'Net'] = s[0]
        df.loc[row[0], 'Type'] = 'Baseline'
        df.loc[row[0], 'Source Adversary'] = s[2] if len(s)==3 else 'nat'
        df.loc[row[0], 'Target Adversary'] = ''
        df.loc[row[0], 'Transfer'] = 'No transfer'
        
df[df.select_dtypes(include=['number']).columns] *= 100
df=np.round(df,1)

In [178]:
idx = ['Type', 'Transfer', 'Source Adversary', 'Target Adversary']
df = df.sort_values(idx)
df.set_index(idx)[['Clean', 'FGSM 0.0625','PGD 0.0625']]

Clean  FGSM 0.0625  \
Type     Transfer       Source Adversary Target Adversary                       
Baseline No transfer    fgsm                                87.1         78.6   
                        nat                                 92.5         20.0   
                        pgd                                 83.9         45.4   
Transfer 12 layers      fgsm             fgsm               87.2         67.5   
                                         nat                89.1         43.3   
                        nat              fgsm               77.3         41.9   
                                         pgd                60.8         20.0   
         all layers     fgsm             fgsm               85.7         72.3   
                                         nat                91.9         23.6   
                        nat              fgsm               88.1         78.4   
                                         pgd                81.5         45.6   
         feat extractor fgsm             fgsm               66.6         42.7   
                                         nat                69.0         23.5   
                        nat              fgsm               69.3         13.1   
                                         pgd                52.2         13.2   

                                                           PGD 0.0625  
Type     Transfer       Source Adversary Target Adversary              
Baseline No transfer    fgsm                                     13.5  
                        nat                                       0.0  
                        pgd                                      36.1  
Transfer 12 layers      fgsm             fgsm                     2.9  
                                         nat                      2.4  
                        nat              fgsm                     0.5  
                                         pgd                      1.0  
         all layers     fgsm             fgsm                     1.5  
                                         nat                      0.0  
                        nat              fgsm                     1.8  
                                         pgd                     39.2  
         feat extractor fgsm             fgsm                     0.2  
                                         nat                      0.0  
                        nat              fgsm                     0.0  
                                         pgd                      0.0

# Black box attacks

In [74]:
js = json.load(open("../experiments_results/attack_results/black_box_results_all_layers.json"))